In [ ]:
#| default_exp learner.learner
#| default_cls_lvl 3

In [ ]:
#| export
from tsfast.data import *
from tsfast.models import *
from tsfast.learner.callbacks import *
from tsfast.learner.losses import *

from fastai.basics import *
from fastai.callback.tracker import *

In [ ]:
from tsfast.datasets.core import *

In [ ]:
dls = create_dls_test(prediction=True).cpu()
model = SimpleRNN(1,1)

# Create Learner Models
Create Learner with different kinds of models with fitting Parameters and regularizations.

In [ ]:
#| export
def get_inp_out_size(dls):
    '''returns input and output size of a timeseries databunch'''
    tup = dls.one_batch()
    inp = tup[0].shape[-1]
    out = tup[1].shape[-1]
    return inp,out

In [ ]:
test_eq(get_inp_out_size(dls),(2,1)) 

## RNN Learner
The Learners include model specific optimizations. Removing the first n_skip samples of the loss function of transient time, greatly improves training stability. In

In [ ]:
#| export
@delegates(SimpleRNN, keep=True)
def RNNLearner(dls,loss_func=nn.L1Loss(),metrics=[fun_rmse],n_skip=0,num_layers=1,hidden_size=100,stateful=False,opt_func=Adam,cbs=None,**kwargs):
    if cbs is None: cbs = []

    inp,out = get_inp_out_size(dls)
    model = SimpleRNN(inp,out,num_layers,hidden_size,stateful=stateful,**kwargs)
  
    skip = partial(SkipNLoss,n_skip=n_skip)
        
    metrics= [skip(f) for f in metrics]

    if stateful: 
        cbs.append(TbpttResetCB())
        # if stateful apply n_skip with a callback for the first minibatch of a tbptt sequence
        cbs.append(SkipFirstNCallback(n_skip))
    else:
        loss_func = skip(loss_func)
        
    lrn = Learner(dls,model,loss_func=loss_func,opt_func=opt_func,metrics=metrics,cbs=cbs,lr=3e-3)
    return lrn

In [ ]:
RNNLearner(dls,rnn_type='gru').fit(1,1e-4)

epoch,train_loss,valid_loss,fun_rmse,time
0,0.162996,0.160148,0.202793,00:00


In [ ]:
RNNLearner(dls,rnn_type='gru',stateful=True).fit(1,1e-4)

epoch,train_loss,valid_loss,fun_rmse,time
0,0.161937,0.172497,0.214227,00:00


In [ ]:
RNNLearner(dls,rnn_type='gru',stateful=True, n_skip=20).fit(1,1e-4)

epoch,train_loss,valid_loss,fun_rmse,time
0,0.198024,0.207973,0.267253,00:00


## TCN Learner
Performs better on multi input data. Higher beta values allow a way smoother prediction. Way faster then RNNs in prediction. 

In [ ]:
#| export
@delegates(TCN, keep=True)
def TCNLearner(dls,num_layers=3,hidden_size=100,loss_func=nn.L1Loss(),metrics=[fun_rmse],n_skip=None,opt_func=Adam,cbs=None,**kwargs):
    inp,out = get_inp_out_size(dls)
    n_skip = 2**num_layers if n_skip is None else n_skip
    model = TCN(inp,out,num_layers,hidden_size,**kwargs)
  
    skip = partial(SkipNLoss,n_skip=n_skip)
        
    metrics= [skip(f) for f in metrics]
    loss_func = skip(loss_func)
        
    lrn = Learner(dls,model,loss_func=loss_func,opt_func=opt_func,metrics=metrics,cbs=cbs,lr=3e-3)
    return lrn

In [ ]:
TCNLearner(dls,num_layers=6,loss_func=nn.L1Loss()).fit(1)

epoch,train_loss,valid_loss,fun_rmse,time
0,0.224532,0.101968,0.128896,00:00


## CRNN Learner

In [ ]:
#| export
@delegates(CRNN, keep=True)
def CRNNLearner(dls,loss_func=nn.L1Loss(),metrics=[fun_rmse],n_skip=0,opt_func=Adam,cbs=None,**kwargs):
    inp,out = get_inp_out_size(dls)
    model = CRNN(inp,out,**kwargs)
  
    skip = partial(SkipNLoss,n_skip=n_skip)
        
    metrics= [skip(f) for f in metrics]
    loss_func = skip(loss_func)
        
    lrn = Learner(dls,model,loss_func=loss_func,opt_func=opt_func,metrics=metrics,cbs=cbs,lr=3e-3)
    return lrn

In [ ]:
CRNNLearner(dls,rnn_type='gru').fit(1,3e-2)

epoch,train_loss,valid_loss,fun_rmse,time
0,0.145227,0.084660,0.121797,00:00


## Autoregressive Learner

In [ ]:
#| export
@delegates(TCN, keep=True)
def AR_TCNLearner(dls,hl_depth=3,alpha=1,beta=1,early_stop=0,metrics=None,n_skip=None,opt_func=Adam,**kwargs):
    n_skip = 2**hl_depth if n_skip is None else n_skip
    skip = partial(SkipNLoss,n_skip=n_skip)
    
    inp,out = get_inp_out_size(dls)
    model = AR_Model(TCN(inp+out,out,hl_depth,**kwargs),ar=False)
    model.init_normalize(dls.one_batch())
    
    cbs=[ARInitCB(),TimeSeriesRegularizer(alpha=alpha,beta=beta,modules=[model.model.conv_layers[-1]])]#SaveModelCallback()
    if early_stop > 0:
        cbs += [EarlyStoppingCallback(patience=early_stop)]
        
    if metrics is None: metrics=SkipNLoss(fun_rmse,n_skip)
        
    lrn = Learner(dls,model,loss_func=nn.L1Loss(),opt_func=opt_func,metrics=metrics,cbs=cbs,lr=3e-3)
    return lrn

In [ ]:
AR_TCNLearner(dls).fit(1)

epoch,train_loss,valid_loss,fun_rmse,time


In [ ]:
#| export
@delegates(SimpleRNN, keep=True)
def AR_RNNLearner(dls,alpha=0,beta=0,early_stop=0,metrics=None,n_skip=0,opt_func=Adam,**kwargs):
    skip = partial(SkipNLoss,n_skip=n_skip)
    
    inp,out = get_inp_out_size(dls)
    model = AR_Model(SimpleRNN(inp+out,out,**kwargs),ar=False)
    model.init_normalize(dls.one_batch())
    
    cbs=[ARInitCB(),TimeSeriesRegularizer(alpha=alpha,beta=beta,modules=[model.model.rnn])]#SaveModelCallback()
    if early_stop > 0:
        cbs += [EarlyStoppingCallback(patience=early_stop)]
        
    if metrics is None: metrics=SkipNLoss(fun_rmse,n_skip)
        
    lrn = Learner(dls,model,loss_func=nn.L1Loss(),opt_func=opt_func,metrics=metrics,cbs=cbs,lr=3e-3)
    return lrn

In [ ]:
AR_RNNLearner(dls).fit(1)

epoch,train_loss,valid_loss,fun_rmse,time
0,0.127089,0.075756,0.095623,00:00


In [ ]:
#| include: false
import nbdev; nbdev.nbdev_export()